## Preparations

Make you you have started a MLFlow application from Apolo Console.

## Installing Dependencies

In [ ]:
!pip install scikit-learn numpy mlflow skl2onnx onnxruntime apolo-sdk apolo-extras

In [ ]:
import mlflow.client
import mlflow.experiments
import mlflow
import mlflow.models
import mlflow.tracking
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
import numpy as np
import argparse
from sklearn.datasets import load_iris
import skl2onnx
import os
import apolo_sdk
import asyncio

from mlflow import (
    get_run,
    start_run,
)


This function uses Apolo SDK to get the current MLFlow instance URL

In [ ]:
async def get_mlflow_url():
    async with apolo_sdk.get() as client:
        jobs = []
        async with client.jobs.list(tags=["job:mlflow"], statuses=[apolo_sdk.JobStatus.RUNNING]) as job_iter:
            jobs += [job async for job in job_iter]
        async with client.jobs.list(tags=["platform-app-type:mlflow"], statuses=[apolo_sdk.JobStatus.RUNNING]) as job_iter:
            jobs += [job async for job in job_iter]
        if len(jobs) > 0:
            print("MLFlow instance found. Logging model metadata")
            return "http://"+str(jobs[0].internal_hostname)
        print("MLFlow instance not found")
        return None

In [ ]:
tracking_uri = os.environ.get("MLFLOW_TRACKING_URI", await get_mlflow_url())
mlflow.set_tracking_uri(tracking_uri)
print(tracking_uri)

Start training and logging the model, its parameters and metrics.

In [ ]:
with start_run() as run:
    print("Logging params")
    mlflow.log_param("param1", "value1")
    mlflow.log_metric("metric1", 1.0)
    print("Setting tag")
    mlflow.set_tag("tag1", "value1")
    
    run_id = run.info.run_id
    fetched_run = get_run(run_id)

    print("Saving artifact")
    content = "artifact content"
    with open("test_artifact.txt", "w") as f:
        f.write(content)
    mlflow.log_artifact("test_artifact.txt")
    
    print("Getting artifact")
    run_id = run.info.run_id
    artifact_uri = mlflow.get_artifact_uri("test_artifact.txt")
    
    print("Traning model")
    data = load_iris()
    X = data.data
    y = data.target
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

    sc = StandardScaler()

    sc.fit(X_train)

    X_train_std = sc.transform(X_train)
    X_test_std = sc.transform(X_test)

    model = Perceptron(max_iter=40, eta0=0.1, random_state=0)
    model.fit(X_train_std, y_train)
    y_pred = model.predict(X_test_std)

    print('Misclassfied samples: %d' % (y_test != y_pred).sum())
    
    signature = mlflow.models.infer_signature(X_train_std, model.predict(X_train_std))
    mlflow.sklearn.log_model(model, "model", signature=signature)
    model_uri = f"runs:/{run.info.run_id}/model"
    print("Registering model")
    registered_model = mlflow.register_model(model_uri, "iris_perceptron")
    
    assert registered_model.name == "iris_perceptron"
    # assert registered_model.version == '1'

    loaded_model = mlflow.sklearn.load_model(f"models:/{registered_model.name}/1")
    prediction = loaded_model.predict([[0.1, 0.5, 0.8, 0.2]])

    # assert isinstance(loaded_model, Perceptron)
    print(loaded_model)
    assert prediction.shape == (1,)
    
    assert fetched_run.data.params["param1"] == "value1"
    assert fetched_run.data.metrics["metric1"] == 1.0

    options = {id(model): {"zipmap": False}}
    onx = skl2onnx.to_onnx(model, X_train_std.astype(np.float32), options=options)
    mlflow.onnx.log_model(onx, "onnx_model", signature=signature)

    print("Registering ONNX model")
    onnx_model_uri = f"runs:/{run.info.run_id}/onnx_model"
    registered_model = mlflow.register_model(onnx_model_uri, "onnx_iris_perceptron")

    client = mlflow.tracking.MlflowClient()
    print("Registered models: ", client.search_registered_models())

## Starting an Inference Server

Go to Apolo Console, start a Apolo Deploy application, select your model from the list and start an inference server.

After you've successfully started an inference server with Apolo Deploy, open the Jupyter Notebook called `inference-demo.ipynb`